In [1]:
import geopandas as gpd
"""
This script performs a biodiversity assessment by analyzing the intersection between fishing zones and various biodiversity layers.
It calculates the total area and percentage of each biodiversity layer within each fishing zone.
The script generates summary reports in CSV format and a PDF report with overall and category-specific statistics.
Additionally, it creates bar charts to visualize the percentage distribution of biodiversity within zones and the percentage of each zone occupied by each biodiversity layer.
Parameters:
    None
Returns:
    None
"""
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from reportlab.lib.pagesizes import A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import os
import logging
import numpy as np
from reportlab.lib import colors

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define paths to grouped layers and their corresponding category column names
grouped_layers = {
    'Abyssal Classification': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Missed\Palau_Seafloor_Abyssal_Classification.shp', 'category_column': 'Class'},
    'Basins': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Basins.shp', 'category_column': 'Type'},
    'Bridges': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Bridges.shp', 'category_column': None},
    'Canyons': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Missed\Palau_Seafloor_Canyons.shp', 'category_column': 'type'},
    'Escarpments': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Missed\Palau___Seafloor_Escarpments.shp', 'category_column': None},
    'Ridges': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Ridges.shp', 'category_column': None},
    'Shelf Classification': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Missed\Palau_Seafloor_Shelf_Classification.shp', 'category_column': 'Class'},
    'Slopes': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Slope.shp', 'category_column': None},
    'Trenches': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Trenches.shp', 'category_column': None},
    'Hadal': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Hadal0.shp', 'category_column': None},
    'Rift Valleys': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Rift_valleys.shp', 'category_column': None},
    'Seamounts': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Seamounts.shp', 'category_column': None},
    'Bioregions': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Missed\Palau_SWP_Deepwater_Bioregions_technical_analysis_final_28.03.18.shp', 'category_column': 'Draft name'},
    'Spreading Ridges': {'path': r'C:\Users\kishank\Downloads\MSP Datasets\Workingfolder\Reprojected\Clipped\Palau__Seafloor_Spreading_ridges0.shp', 'category_column': None},
}

fishing_zone_layer_path = r"C:\Users\kishank\Downloads\Zones_Palau.shp"

# Load fishing zone layer
logging.info("Loading fishing zone layer")
fishing_zone_layer = gpd.read_file(fishing_zone_layer_path).to_crs(epsg=3857)  # Reproject to a suitable projected CRS

# Ensure output directory exists
output_directory = r'C:\Users\kishank\Downloads\MSP Datasets\Reprojected\Clipped\Missed4\intersect_outputs'
os.makedirs(output_directory, exist_ok=True)

# Iterate through each unique zone
for zone in fishing_zone_layer['Zone'].unique():
    logging.info(f"Processing Zone: {zone}")
    zone_layer = fishing_zone_layer[fishing_zone_layer['Zone'] == zone]
    zone_name = f"{zone}"

    # Calculate the total area of the fishing zone
    fishing_zone_total_area = zone_layer['geometry'].area.sum() / 1e6  # Convert to square kilometers

    # Initialize DataFrames for summary reports
    overall_summary = []
    category_summary = []

    total_layers = len(grouped_layers)
    processed_layers = 0

    for group, layer_info in grouped_layers.items():
        processed_layers += 1
        progress = (processed_layers / total_layers) * 100
        logging.info(f"Processing {group} ({processed_layers}/{total_layers}) - {progress:.2f}% completed")

        layer_path = layer_info['path']
        category_column = layer_info['category_column']

        biodiversity_layer = gpd.read_file(layer_path).to_crs(epsg=3857)  # Reproject to a suitable projected CRS
        if biodiversity_layer.empty:
            logging.error(f"Failed to load {layer_path}")
            continue

        # Perform intersection
        intersected_layer = gpd.overlay(biodiversity_layer, zone_layer, how='intersection')

        # Calculate areas within zone
        intersected_layer['area_zone'] = intersected_layer['geometry'].area / 1e6  # Convert to square kilometers
        biodiversity_layer['total_area'] = biodiversity_layer['geometry'].area / 1e6  # Convert to square kilometers

        # Save intersected spatial files
        intersected_layer_path = os.path.join(output_directory, f'{group}_{zone_name}_intersected.shp')
        intersected_layer.to_file(intersected_layer_path)

        # Summarize overall statistics
        total_area = biodiversity_layer['total_area'].sum()
        area_within_zone = intersected_layer['area_zone'].sum()
        percentage_within_zone = (area_within_zone / total_area) * 100 if total_area > 0 else 0
        percentage_of_fishing_zone = (area_within_zone / fishing_zone_total_area) * 100 if fishing_zone_total_area > 0 else 0
        overall_summary.append([group, total_area, area_within_zone, percentage_within_zone, percentage_of_fishing_zone])

        if category_column and category_column in biodiversity_layer.columns:
            # Summarize category-specific statistics
            for category in biodiversity_layer[category_column].unique():
                category_biodiversity = biodiversity_layer[biodiversity_layer[category_column] == category]
                category_intersected = intersected_layer[intersected_layer[category_column] == category]

                category_total_area = category_biodiversity['total_area'].sum()
                category_area_within_zone = category_intersected['area_zone'].sum()
                category_percentage_within_zone = (category_area_within_zone / category_total_area) * 100 if category_total_area > 0 else 0
                category_percentage_of_fishing_zone = (category_area_within_zone / fishing_zone_total_area) * 100 if fishing_zone_total_area > 0 else 0
                category_summary.append([group, category, category_total_area, category_area_within_zone, category_percentage_within_zone, category_percentage_of_fishing_zone])

                # Generate bar chart
                categories_list = biodiversity_layer[category_column].unique()
                total_areas = [biodiversity_layer[biodiversity_layer[category_column] == cat]['total_area'].sum() for cat in categories_list]
                within_zone_areas = [intersected_layer[intersected_layer[category_column] == cat]['area_zone'].sum() for cat in categories_list]
                total_areas_percentage = [total_area / sum(total_areas) * 100 for total_area in total_areas]
                within_zone_areas_percentage = [within_zone_area / sum(within_zone_areas) * 100 for within_zone_area in within_zone_areas]

                plt.figure(figsize=(15, 8))  # Adjusted figure size for better readability
                plt.bar(categories_list, total_areas_percentage, label='Total Area Percentage', alpha=0.6, color='b')
                plt.bar(categories_list, within_zone_areas_percentage, label='Area within Zones Percentage', alpha=0.6, color='orange')
                plt.xlabel('Category')
                plt.ylabel('Percentage (%)')
                plt.title(f'Percentage Distribution for {group}')
                plt.gca().yaxis.set_major_formatter(ScalarFormatter(useOffset=False))  # Disable scientific notation
                if group == 'Bioregions':
                    plt.xticks(rotation=90, ha='right')  # Rotate x-axis labels for bioregions to avoid overlap
                else:
                    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
                plt.legend()
                plt.tight_layout()  # Adjust layout to prevent label cutoff
                plt.savefig(os.path.join(output_directory, f'{group}_{zone_name}_percentage_distribution.png'))
                plt.close()

                # Generate bar chart for percentage of fishing zone
                plt.figure(figsize=(15, 8))  # Adjusted figure size for better readability
                plt.bar(categories_list, [within_zone / fishing_zone_total_area * 100 for within_zone in within_zone_areas], label='Percentage of Zone', alpha=0.6, color='g')
                plt.xlabel('Category')
                plt.ylabel('Percentage of Zone (%)')
                plt.title(f'Percentage of Zone Occupied by {group}')
                plt.gca().yaxis.set_major_formatter(ScalarFormatter(useOffset=False))  # Disable scientific notation
                if group == 'Bioregions':
                    plt.xticks(rotation=90, ha='right')  # Rotate x-axis labels for bioregions to avoid overlap
                else:
                    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
                plt.legend()
                plt.tight_layout()  # Adjust layout to prevent label cutoff
                plt.savefig(os.path.join(output_directory, f'{group}_{zone_name}_zone_percentage.png'))
                plt.close()
        elif category_column:
            logging.error(f"Category column '{category_column}' does not exist in {layer_path}")

    # Create summary DataFrames
    overall_summary_df = pd.DataFrame(overall_summary, columns=['Biodiversity Layer', 'Total Area (sq.km)', 'Area within Zone (sq.km)', 'Percentage within Zone (%)', 'Percentage of Zone (%)'])
    category_summary_df = pd.DataFrame(category_summary, columns=['Layer', 'Category', 'Total Area (sq.km)', 'Area within Zone (sq.km)', 'Percentage within Zone (%)', 'Percentage of Zone (%)'])

    # Format data to avoid scientific notation
    pd.options.display.float_format = '{:.6f}'.format

    # Save summaries to CSV
    overall_summary_df.to_csv(os.path.join(output_directory, f'overall_summary_{zone_name}.csv'), index=False)
    category_summary_df.to_csv(os.path.join(output_directory, f'category_summary_{zone_name}.csv'), index=False)

    # Validate calculations
    for group, layer_info in grouped_layers.items():
        layer_path = layer_info['path']
        category_column = layer_info['category_column']

        biodiversity_layer = gpd.read_file(layer_path).to_crs(epsg=3857)
        intersected_layer = gpd.read_file(os.path.join(output_directory, f'{group}_{zone_name}_intersected.shp'))

        calculated_area_within_zone = intersected_layer['geometry'].area.sum() / 1e6
        summary_area_within_zone = category_summary_df[category_summary_df['Layer'] == group]['Area within Zone (sq.km)'].sum()

        if not np.isclose(calculated_area_within_zone, summary_area_within_zone, rtol=1e-5):
            logging.error(f"Validation error for {group}: Calculated area within zone ({calculated_area_within_zone}) does not match summary area ({summary_area_within_zone})")

    # PDF report generation
    logging.info("Generating PDF report")
    report_path = os.path.join(output_directory, f'Biodiversity_Assessment_Report_{zone_name}.pdf')
    doc = SimpleDocTemplate(report_path, pagesize=A3, rightMargin=30, leftMargin=30, topMargin=30, bottomMargin=18)

    elements = []
    styles = getSampleStyleSheet()

    # Title
    title = Paragraph(f"Biodiversity Assessment Report - {zone_name}", styles['Title'])
    elements.append(title)
    elements.append(Spacer(1, 12))

    # Overall Summary Table
    overall_summary_title = Paragraph("Overall Summary", styles['Heading2'])
    elements.append(overall_summary_title)
    overall_data = [overall_summary_df.columns.tolist()] + overall_summary_df.values.tolist()
    overall_table = Table(overall_data, repeatRows=1)
    overall_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(overall_table)
    elements.append(Spacer(1, 12))

    # Category Summary Table
    category_summary_title = Paragraph("Category Summary", styles['Heading2'])
    elements.append(category_summary_title)
    category_data = [category_summary_df.columns.tolist()] + category_summary_df.values.tolist()

    # Calculate column widths based on the number of columns
    num_cols = len(category_summary_df.columns)
    page_width = A3[0] - doc.leftMargin - doc.rightMargin
    col_width = page_width / num_cols

    category_table = Table(category_data, colWidths=[col_width] * num_cols, repeatRows=1)
    category_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0, 0), (-1, -1), 3),
        ('RIGHTPADDING', (0, 0), (-1, -1), 3),
    ]))
    elements.append(category_table)
    elements.append(Spacer(1, 12))

    # Add images
    for group in grouped_layers.keys():
        image_path = os.path.join(output_directory, f'{group}_{zone_name}_percentage_distribution.png')
        if os.path.exists(image_path):
            elements.append(Paragraph(f'Percentage Distribution for {group}', styles['Heading3']))
            elements.append(Image(image_path, width=700, height=400))
            elements.append(Spacer(1, 12))

        image_path = os.path.join(output_directory, f'{group}_{zone_name}__zone_percentage.png')
        if os.path.exists(image_path):
            elements.append(Paragraph(f'Percentage of Zone Occupied by {group}', styles['Heading3']))
            elements.append(Image(image_path, width=700, height=400))
            elements.append(Spacer(1, 12))

    doc.build(elements)
    logging.info(f"Processing for {zone_name} completed")

logging.info("All zones processed")


2024-09-16 12:50:18,087 - INFO - Loading fishing zone layer
2024-09-16 12:50:18,183 - ERROR - PROJ: proj_identify: C:\OSGeo4W\share\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
2024-09-16 12:50:18,200 - ERROR - PROJ: proj_identify: C:\OSGeo4W\share\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
2024-09-16 12:50:18,241 - INFO - Processing Zone: New PNMS Footprint
2024-09-16 12:50:18,241 - INFO - Processing Abyssal Classification (1/14) - 7.14% completed
2024-09-16 12:50:18,250 - ERROR - PROJ: proj_identify: C:\OSGeo4W\share\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
2024-09-16 12:50:18,274 - ERROR - PROJ: proj_identify: C:\OSGeo4W\share\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It c